# 📊 Analyse des Dividendes - Top Actions des 10 Dernières Années

## Objectifs du Projet
1. **Analyser** les dividendes des actions les plus performantes sur 10 ans
2. **Comparer** les performances et identifier les meilleures opportunités
3. **Prédire** les meilleures actions pour les 10 prochaines années
4. **Utiliser** des techniques de Machine Learning avancées (XGBoost, Cross-Validation)

## Technologies Utilisées
- **API**: yfinance (données financières Yahoo Finance)
- **Analyse**: Pandas, NumPy
- **Visualisation**: Seaborn, Matplotlib, Plotly
- **Machine Learning**: Scikit-learn, XGBoost
- **Optimisation**: Optuna (Bayesian Optimization)

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import yfinance as yf
from datetime import datetime, timedelta
import time

# Machine Learning
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.base import clone

# XGBoost
import xgboost as xgb

# Optimisation
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configuration des graphiques
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Toutes les bibliothèques ont été importées avec succès!")

## 2. Configuration avec yfinance

yfinance utilise les données de Yahoo Finance gratuitement et sans clé API. Cette approche est plus simple et plus fiable.

In [ ]:
# Configuration des actions à analyser (Top S&P 500 avec dividendes)
DIVIDEND_ARISTOCRATS = [
    'JNJ',   # Johnson & Johnson
    'PG',    # Procter & Gamble
    'KO',    # Coca-Cola
    'PEP',   # PepsiCo
    'ABBV',  # AbbVie
    'MRK',   # Merck
    'XOM',   # Exxon Mobil
    'CVX',   # Chevron
    'T',     # AT&T
    'VZ',    # Verizon
    'IBM',   # IBM
    'CSCO',  # Cisco
    'INTC',  # Intel
    'WMT',   # Walmart
    'HD'     # Home Depot
]

# Configuration des périodes
START_DATE = "2014-01-01"  # 10 ans de données
END_DATE = datetime.now().strftime("%Y-%m-%d")

print(f"📊 Nombre d'actions à analyser: {len(DIVIDEND_ARISTOCRATS)}")
print(f"🏢 Actions sélectionnées: {', '.join(DIVIDEND_ARISTOCRATS)}")
print(f"📅 Période d'analyse: {START_DATE} à {END_DATE}")

## 3. Fonctions de Collecte de Données avec yfinance

Nous créons des fonctions robustes pour récupérer les données depuis Yahoo Finance via yfinance.

In [ ]:
class YFinanceClient:
    """Client pour interagir avec yfinance (Yahoo Finance)"""
    
    def __init__(self):
        self.cache = {}
        
    def get_stock_data(self, symbol, start_date, end_date):
        """Récupère les données historiques avec dividendes"""
        
        # Vérifier le cache
        cache_key = f"{symbol}_{start_date}_{end_date}"
        if cache_key in self.cache:
            print(f"✅ {symbol} - Données récupérées depuis le cache")
            return self.cache[cache_key]
        
        try:
            # Créer l'objet ticker
            ticker = yf.Ticker(symbol)
            
            # Récupérer les données historiques
            df = ticker.history(start=start_date, end=end_date, auto_adjust=False)
            
            if not df.empty:
                # Renommer les colonnes pour correspondre au format précédent
                df.columns = df.columns.str.lower()
                
                # Ajouter les dividendes séparément si nécessaire
                dividends = ticker.dividends
                if not dividends.empty:
                    # Filtrer les dividendes sur la période
                    dividends = dividends[(dividends.index >= start_date) & (dividends.index <= end_date)]
                    
                    # Ajouter une colonne dividend_amount initialisée à 0
                    df['dividend_amount'] = 0.0
                    
                    # Mettre à jour avec les vraies valeurs de dividendes
                    for date, dividend in dividends.items():
                        if date in df.index:
                            df.loc[date, 'dividend_amount'] = dividend
                
                # Stocker dans le cache
                self.cache[cache_key] = df
                print(f"✅ {symbol} - Données téléchargées avec succès ({len(df)} jours)")
                return df
            else:
                print(f"❌ Aucune donnée disponible pour {symbol}")
                return None
                
        except Exception as e:
            print(f"❌ Erreur pour {symbol}: {str(e)}")
            return None
    
    def get_company_info(self, symbol):
        """Récupère les informations fondamentales de l'entreprise"""
        
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            
            if info and 'symbol' in info:
                print(f"✅ {symbol} - Informations récupérées")
                return info
            else:
                print(f"❌ Pas d'informations disponibles pour {symbol}")
                return None
                
        except Exception as e:
            print(f"❌ Erreur pour {symbol}: {str(e)}")
            return None

# Initialiser le client
client = YFinanceClient()
print("✅ Client yfinance initialisé")

## 4. Collecte des Données Historiques

Nous allons maintenant télécharger les données pour toutes les actions sélectionnées avec yfinance.

✅ **Avantage**: Pas de limite d'API, téléchargement plus rapide.

In [ ]:
# Collecte des données pour toutes les actions
stock_data = {}
company_info = {}

print("🔄 Début du téléchargement des données...\n")

for i, symbol in enumerate(DIVIDEND_ARISTOCRATS):
    print(f"[{i+1}/{len(DIVIDEND_ARISTOCRATS)}] Téléchargement de {symbol}...")
    
    # Données historiques
    df = client.get_stock_data(symbol, START_DATE, END_DATE)
    if df is not None:
        stock_data[symbol] = df
    
    # Informations de l'entreprise
    info = client.get_company_info(symbol)
    if info is not None:
        company_info[symbol] = info

print(f"\n✅ Téléchargement terminé!")
print(f"📊 Actions téléchargées: {len(stock_data)}/{len(DIVIDEND_ARISTOCRATS)}")
print(f"📋 Informations d'entreprises: {len(company_info)}/{len(DIVIDEND_ARISTOCRATS)}")

# Afficher un résumé des données collectées
if stock_data:
    print(f"\n📈 Résumé des données:")
    for symbol, df in stock_data.items():
        total_dividends = df['dividend_amount'].sum() if 'dividend_amount' in df.columns else 0
        print(f"  {symbol}: {len(df)} jours, Dividendes totaux: ${total_dividends:.2f}")

In [7]:
# Test avec les imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import yfinance as yf
from datetime import datetime, timedelta
import time

print("✅ Toutes les bibliothèques ont été importées avec succès!")

✅ Toutes les bibliothèques ont été importées avec succès!


In [8]:
# Configuration des actions à analyser (Top S&P 500 avec dividendes)
DIVIDEND_ARISTOCRATS = [
    'JNJ',   # Johnson & Johnson
    'PG',    # Procter & Gamble
    'KO',    # Coca-Cola
    'PEP',   # PepsiCo
    'ABBV',  # AbbVie
    'MRK',   # Merck
    'XOM',   # Exxon Mobil
    'CVX',   # Chevron
    'T',     # AT&T
    'VZ',    # Verizon
    'IBM',   # IBM
    'CSCO',  # Cisco
    'INTC',  # Intel
    'WMT',   # Walmart
    'HD'     # Home Depot
]

# Configuration des périodes
START_DATE = "2014-01-01"  # 10 ans de données
END_DATE = datetime.now().strftime("%Y-%m-%d")

print(f"📊 Nombre d'actions à analyser: {len(DIVIDEND_ARISTOCRATS)}")
print(f"🏢 Actions sélectionnées: {', '.join(DIVIDEND_ARISTOCRATS)}")
print(f"📅 Période d'analyse: {START_DATE} à {END_DATE}")

📊 Nombre d'actions à analyser: 15
🏢 Actions sélectionnées: JNJ, PG, KO, PEP, ABBV, MRK, XOM, CVX, T, VZ, IBM, CSCO, INTC, WMT, HD
📅 Période d'analyse: 2014-01-01 à 2025-05-26


In [9]:
class YFinanceClient:
    """Client pour interagir avec yfinance (Yahoo Finance)"""
    
    def __init__(self):
        self.cache = {}
        
    def get_stock_data(self, symbol, start_date, end_date):
        """Récupère les données historiques avec dividendes"""
        
        # Vérifier le cache
        cache_key = f"{symbol}_{start_date}_{end_date}"
        if cache_key in self.cache:
            print(f"✅ {symbol} - Données récupérées depuis le cache")
            return self.cache[cache_key]
        
        try:
            # Créer l'objet ticker
            ticker = yf.Ticker(symbol)
            
            # Récupérer les données historiques
            df = ticker.history(start=start_date, end=end_date, auto_adjust=False)
            
            if not df.empty:
                # Renommer les colonnes pour correspondre au format précédent
                df.columns = df.columns.str.lower()
                
                # Ajouter les dividendes séparément si nécessaire
                dividends = ticker.dividends
                if not dividends.empty:
                    # Filtrer les dividendes sur la période
                    dividends = dividends[(dividends.index >= start_date) & (dividends.index <= end_date)]
                    
                    # Ajouter une colonne dividend_amount initialisée à 0
                    df['dividend_amount'] = 0.0
                    
                    # Mettre à jour avec les vraies valeurs de dividendes
                    for date, dividend in dividends.items():
                        if date in df.index:
                            df.loc[date, 'dividend_amount'] = dividend
                
                # Stocker dans le cache
                self.cache[cache_key] = df
                print(f"✅ {symbol} - Données téléchargées avec succès ({len(df)} jours)")
                return df
            else:
                print(f"❌ Aucune donnée disponible pour {symbol}")
                return None
                
        except Exception as e:
            print(f"❌ Erreur pour {symbol}: {str(e)}")
            return None
    
    def get_company_info(self, symbol):
        """Récupère les informations fondamentales de l'entreprise"""
        
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            
            if info and 'symbol' in info:
                print(f"✅ {symbol} - Informations récupérées")
                return info
            else:
                print(f"❌ Pas d'informations disponibles pour {symbol}")
                return None
                
        except Exception as e:
            print(f"❌ Erreur pour {symbol}: {str(e)}")
            return None

# Initialiser le client
client = YFinanceClient()
print("✅ Client yfinance initialisé")

✅ Client yfinance initialisé


In [10]:
# Test avec quelques actions d'abord
test_symbols = ['KO', 'JNJ', 'PG']  # Test avec 3 actions
stock_data = {}
company_info = {}

print("🔄 Test du téléchargement avec yfinance...\n")

for i, symbol in enumerate(test_symbols):
    print(f"[{i+1}/{len(test_symbols)}] Test de {symbol}...")
    
    # Données historiques
    df = client.get_stock_data(symbol, START_DATE, END_DATE)
    if df is not None:
        stock_data[symbol] = df
    
    # Informations de l'entreprise  
    info = client.get_company_info(symbol)
    if info is not None:
        company_info[symbol] = info

print(f"\n✅ Test terminé!")
print(f"📊 Actions téléchargées: {len(stock_data)}/{len(test_symbols)}")
print(f"📋 Informations d'entreprises: {len(company_info)}/{len(test_symbols)}")

# Afficher un résumé des données collectées
if stock_data:
    print(f"\n📈 Résumé des données:")
    for symbol, df in stock_data.items():
        total_dividends = df['dividend_amount'].sum() if 'dividend_amount' in df.columns else 0
        print(f"  {symbol}: {len(df)} jours, Dividendes totaux: ${total_dividends:.2f}")
        print(f"    Colonnes disponibles: {list(df.columns)}")
        print(f"    Période: {df.index.min()} à {df.index.max()}")
        print()

🔄 Test du téléchargement avec yfinance...

[1/3] Test de KO...
✅ KO - Données téléchargées avec succès (2866 jours)
✅ KO - Informations récupérées
[2/3] Test de JNJ...
✅ JNJ - Données téléchargées avec succès (2866 jours)
✅ JNJ - Informations récupérées
[3/3] Test de PG...
✅ PG - Données téléchargées avec succès (2866 jours)
✅ PG - Informations récupérées

✅ Test terminé!
📊 Actions téléchargées: 3/3
📋 Informations d'entreprises: 3/3

📈 Résumé des données:
  KO: 2866 jours, Dividendes totaux: $17.95
    Colonnes disponibles: ['open', 'high', 'low', 'close', 'adj close', 'volume', 'dividends', 'stock splits', 'dividend_amount']
    Période: 2014-01-02 00:00:00-05:00 à 2025-05-23 00:00:00-04:00

  JNJ: 2866 jours, Dividendes totaux: $42.94
    Colonnes disponibles: ['open', 'high', 'low', 'close', 'adj close', 'volume', 'dividends', 'stock splits', 'dividend_amount']
    Période: 2014-01-02 00:00:00-05:00 à 2025-05-23 00:00:00-04:00

  PG: 2866 jours, Dividendes totaux: $36.27
    Colonne